<img align=right src=images/HashiCorp_PrimaryLogo_Black_RGB.png width=150>
<img src=images/Acme.jpeg width=100 align="left">

# AWS Dynamic Credentials
---

## Business Value of Capability
The AWS secrets engine generates AWS access credentials dynamically based on IAM policies. This generally makes working with AWS IAM easier, since it does not involve clicking in the web UI. Additionally, the process is codified and mapped to internal auth methods (such as LDAP). The AWS IAM credentials are time-based and are automatically revoked when the Vault lease expires.
### Setup
---

In [4]:
export VAULT_ADDR=http://127.0.0.1:8200
export VAULT_SKIP_VERIFY=true

In [5]:
vault login root

WARNING! The VAULT_TOKEN environment variable is set! This takes precedence
over the value set by this command. To use the value set by this command,
unset the VAULT_TOKEN environment variable or set it to the token displayed
below.

Success! You are now authenticated. The token information displayed below
is already stored in the token helper. You do NOT need to run "vault login"
again. Future Vault requests will automatically use this token.

Key                  Value
---                  -----
token                root
token_accessor       DX9mkWrAfEN0wRJaMBCuy6hN
token_duration       ∞
token_renewable      false
token_policies       ["root"]
identity_policies    []
policies             ["root"]


In [6]:
vault status

Key             Value
---             -----
Seal Type       shamir
Initialized     true
Sealed          false
Total Shares    1
Threshold       1
Version         1.6.0
Storage Type    inmem_transactional_ha
Cluster Name    vault-cluster-6a9fd92b
Cluster ID      ad4f5b1b-962e-cf1a-33c8-1ba9651d5fa0
HA Enabled      true
HA Cluster      https://127.0.0.1:8201
HA Mode         active


In [ ]:
vault -version

### Demo
---
**Enable the Vault AWS secrets engine**

In [ ]:
vault secrets enable aws

**Write some secrets to Vault**

In [ ]:
vault write aws/config/root \
      access_key=$AWS_ACCESS_KEY \
      secret_key=$AWS_SECRET_ACCESS_KEY

**Write a role to Vault**

In [ ]:
vault write aws/roles/my-role \
        credential_type=iam_user \
        policy_document=-<<EOF
{
  "Version": "2012-10-17",
  "Statement": [
    {
      "Sid": "Stmt1426528957000",
      "Effect": "Allow",
      "Action": [
        "ec2:*"
      ],
      "Resource": [
        "*"
      ]
    }
  ]
}
EOF

**Reading the role creates the credentials in AWS as an IAM user with those permissions above**

In [ ]:
vault read aws/creds/my-role

In [ ]:
export AWS_ACCESS_KEY_ID=foo
export AWS_SECRET_ACCESS_KEY=bar
aws ec2 describe-regions --filters "Name=endpoint,Values=*us*"

**Revoking the lease removes the user**

In [ ]:
vault lease revoke aws/creds/my-role/<foo>

**Revoke all leases**

In [ ]:
vault lease revoke -prefix=true aws/creds/my-role

**END**